# Comparison Using LFW Dataset - LBPH

Several feature extractor:
1. Face Embedding: Facenet
2. Face Embedding: VGG Face
3. Face Embedding: VGG Face - VGG16
4. Face Embedding: VGG Face - RESNET50
5. LBPH (Local Binary Pattern Histogram)

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Read Dataset

In [2]:
import os

In [3]:
DIR_PATH = '../lfw/'
image_path_list = []
labels = []
name_dictionary = {}
counter = 0
for root, dirs, files in os.walk(DIR_PATH):
    for filename in files:
        person_name = ' '.join(filename.split('.')[0].split('_')[0:-1]) 
        file_path = os.path.join(root, filename)
        if person_name not in name_dictionary:
            counter += 1
            name_dictionary[person_name] = counter
        image_path_list.append(file_path)
        labels.append(name_dictionary[person_name])

In [4]:
print(len(labels))
print(len(image_path_list))

13233
13233


In [5]:
image_path_list[0:5]

['../lfw/Ryan_Newman/Ryan_Newman_0001.jpg',
 '../lfw/Dimitar_Berbatov/Dimitar_Berbatov_0001.jpg',
 '../lfw/Ed_Rendell/Ed_Rendell_0001.jpg',
 '../lfw/Joe_Crede/Joe_Crede_0001.jpg',
 '../lfw/Norman_Mailer/Norman_Mailer_0001.jpg']

## Shuffle It

In [6]:
import random

In [7]:
temp = list(zip(image_path_list, labels))
random.Random(0).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [8]:
image_path_list[0:5]

('../lfw/Tommy_Maddox/Tommy_Maddox_0001.jpg',
 '../lfw/David_Millar/David_Millar_0001.jpg',
 '../lfw/Gregg_Popovich/Gregg_Popovich_0004.jpg',
 '../lfw/Shimon_Peres/Shimon_Peres_0001.jpg',
 '../lfw/Rudolph_Giuliani/Rudolph_Giuliani_0024.jpg')

In [9]:
labels[0:5]

(3038, 1517, 3115, 834, 1016)

In [10]:
name_dictionary['John Ashcroft']

482

# Experiment

In [11]:
FACENET_MODEL_PATH = 'model_data/facenet/20180402-114759'
VGGFACE_MODEL_PATH = 'model_data/vgg_face_weights.h5'

In [12]:
mtcnn_detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
def predict(image_representation_list, true_label, test_representation, min_distance):
    minimum_label = None
    minimum_distance = min_distance
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [14]:
def compute_score(predictions, labels):
    count_same = 0
    for idx, prediction in enumerate(predictions):
        if labels[idx] == prediction:
            count_same += 1
    return count_same / len(predictions)

## LBPH

Based on experiment LBPH have distance below 95 for euclidean if two image are the same person

In [15]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='lbph')

### Testing Starts Here

In [16]:
THRESHOLD = 2.75

In [17]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [18]:
%%time
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0


/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/facerec/feature.py:266: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  H = np.histogram(C, bins=2**self.lbp_operator.neighbors, range=(0, 2**self.lbp_operator.neighbors), normed=True)[0]


Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
CPU times: user 1h 10min 3s, sys: 1h 10min 48s, total: 2h 20min 51s
Wall time: 26min 4s


In [19]:
feature_extractor.extract_image(img)

array([0.06122449, 0.        , 0.        , ..., 0.        , 0.        ,
       0.20408163])

In [20]:
image_path_list[37]

'../lfw/George_W_Bush/George_W_Bush_0233.jpg'

In [21]:
compute_score(prediction_result, labels)

0.05561852943399078

In [22]:
len(prediction_result)

13233

In [23]:
len(labels)

13233

In [24]:
# feature_extractor.extractor.close_session()

In [25]:
image_path_list = sorted(image_path_list)

In [26]:
image_path_list[31:50]

['../lfw/Abdullah_Gul/Abdullah_Gul_0001.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0002.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0003.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0004.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0005.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0006.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0007.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0008.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0009.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0010.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0011.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0012.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0013.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0014.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0015.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0016.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0017.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0018.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0019.jpg']

In [27]:
%%time
for idx, image_path in enumerate(image_path_list[31:32]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
CPU times: user 282 ms, sys: 406 ms, total: 688 ms
Wall time: 125 ms


In [28]:
base_feature = feature_test.copy()
base_feature

array([0.14285714, 0.        , 0.        , ..., 0.        , 0.02040816,
       0.10204082])

In [29]:
%%time
for idx, image_path in enumerate(image_path_list[31:50]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)

Checkpoint 0
0.0
2.9142971474079737
2.8610760100051644
2.990335676011472
2.605220146487012
2.884851067380475
2.110247502809888
2.7962907542863435
3.209471613570895
2.709294167392015
2.4618708326530974
2.204459529206657
2.2606128989466105
2.664496548610891
2.4674473739285787
2.520552751367873
2.260981346980831
2.744575506437612
2.3411641468897937
CPU times: user 5.64 s, sys: 5.53 s, total: 11.2 s
Wall time: 1.96 s


In [30]:
%%time
scores = []
for idx, image_path in enumerate(image_path_list[0:30]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)
        scores.append(distance)
print("AVG:", np.mean(scores))

Checkpoint 0
2.886005816177798
2.985735907408424
2.7512444649266
2.787190309416986
2.970632322110818
2.8477981293897807
2.653846037766684
2.750184575987431
2.5215439892975327
2.7285950878952274
2.7859946035235135
2.6735472803733065
2.917867803685325
2.5845148064333183
2.566889417719204
2.684895353092846
2.6179784017672776
2.8708127793641935
2.5185691051387558
2.4481291040209023
2.7476088578347193
2.98949988086513
2.7509416811844267
2.785994603523514
2.7906251100407005
2.918153267444134
3.0523681413460007
2.962911093688864
2.8644222137996325
2.520387507144968
AVG: 2.764496255078934
CPU times: user 8.97 s, sys: 8.42 s, total: 17.4 s
Wall time: 3.1 s
